<a href="https://colab.research.google.com/github/Otter2022/BlueBookScraper/blob/origin/FirstNotebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#getting data
from google.colab import drive
drive.mount('/content/drive')
basepath = '/content/drive/MyDrive/AntlerHackathon/Data'

Mounted at /content/drive


In [ ]:
#imports
import pandas as pd
import numpy as np
import torch
import sklearn.model_selection

In [ ]:
#Data imports
countries = [
    "Germany", "France", "Spain", "England", "Italy", "Netherlands", "Portugal",
    "Belgium", "Croatia", "Denmark", "Switzerland", "Sweden", "Poland", "Austria",
    "Serbia", "Norway", "Brazil", "Argentina", "Uruguay", "Colombia", "Chile",
    "Ecuador", "Peru", "Mexico", "United States", "Canada", "Costa Rica", "Honduras",
    "Panama", "Nigeria", "Senegal", "Cameroon", "Egypt", "Ghana", "Morocco",
    "Algeria", "Tunisia", "Ivory Coast", "Japan", "South Korea", "Iran", "Australia",
    "Saudi Arabia", "Qatar", "United Arab Emirates", "Iraq", "China PR", "New Zealand", "Bosnia and Herzegovina"
]

# Assign group labels
groups = {
    'Group A': ['United States', 'Germany', 'Nigeria', 'Japan'],
    'Group B': ['Mexico', 'France', 'Senegal', 'South Korea'],
    'Group C': ['Canada', 'Brazil', 'Egypt', 'Iran'],
    'Group D': ['Argentina', 'England', 'Morocco', 'Australia'],
    'Group E': ['Spain', 'Colombia', 'Cameroon', 'Saudi Arabia'],
    'Group F': ['Netherlands', 'Uruguay', 'Ghana', 'Qatar'],
    'Group G': ['Italy', 'Croatia', 'Algeria', 'United Arab Emirates'],
    'Group H': ['Belgium', 'Portugal', 'Tunisia', 'Iraq'],
    'Group I': ['Denmark', 'Switzerland', 'Ivory Coast', 'China PR'],
    'Group J': ['Sweden', 'Ecuador', 'Mali', 'Panama'],
    'Group K': ['Poland', 'Peru', 'South Africa', 'New Zealand'],
    'Group L': ['Norway', 'Chile', 'Costa Rica', 'Bosnia and Herzegovina']
}

shootouts_path = basepath + '/shootouts.csv'
shootouts = pd.read_csv(shootouts_path)
filtered_shootouts = shootouts[shootouts['home_team'].isin(countries)]

results_path = basepath + '/results.csv'
results = pd.read_csv(results_path)
filtered_results = results[results['home_team'].isin(countries)]

goalscorers = basepath + '/goalscorers.csv'
goalscorers = pd.read_csv(goalscorers)
filtered_goalscorers = goalscorers[goalscorers['home_team'].isin(countries)]




# Create a DataFrame with team and group information
team_groups = []
for group, teams in groups.items():
    for team in teams:
        team_groups.append((team, group))

df_teams = pd.DataFrame(team_groups, columns=['Team', 'Group'])
print(df_teams)

# Display the first few rows of the DataFrame
print(df_teams.head())
print(shootouts.head())
print(results.head())
print(goalscorers.head())

print(pd.unique(results['home_team']))

                      Team    Group
0            United States  Group A
1                  Germany  Group A
2                  Nigeria  Group A
3                    Japan  Group A
4                   Mexico  Group B
5                   France  Group B
6                  Senegal  Group B
7              South Korea  Group B
8                   Canada  Group C
9                   Brazil  Group C
10                   Egypt  Group C
11                    Iran  Group C
12               Argentina  Group D
13                 England  Group D
14                 Morocco  Group D
15               Australia  Group D
16                   Spain  Group E
17                Colombia  Group E
18                Cameroon  Group E
19            Saudi Arabia  Group E
20             Netherlands  Group F
21                 Uruguay  Group F
22                   Ghana  Group F
23                   Qatar  Group F
24                   Italy  Group G
25                 Croatia  Group G
26                 Algeria  

In [ ]:
filtered_shootouts.head()

,date,home_team,away_team,winner,first_shooter
1,1971-11-14,South Korea,Vietnam Republic,South Korea,NaN
2,1972-05-07,South Korea,Iraq,Iraq,NaN
5,1973-04-21,Senegal,Ghana,Ghana,NaN
13,1973-08-23,Algeria,Syria,Syria,NaN
14,1973-08-25,Algeria,Iraq,Algeria,NaN


In [ ]:
goalscorers.head()

,date,home_team,away_team,team,scorer,minute,own_goal,penalty
0,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,44.0,False,False
1,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,55.0,False,False
2,1916-07-02,Chile,Uruguay,Uruguay,Isabelino Gradín,70.0,False,False
3,1916-07-02,Chile,Uruguay,Uruguay,José Piendibene,75.0,False,False
4,1916-07-06,Argentina,Chile,Argentina,Alberto Ohaco,2.0,False,False


In [ ]:
results.head()

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False


In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd

# Calculate win frequency for each team based on historical matchups
# Assuming 'filtered_results' contains historical match data

# Calculate number of wins for each team
wins = filtered_results[filtered_results['home_score'] > filtered_results['away_score']].groupby('home_team').size().reset_index(name='wins')
losses = filtered_results[filtered_results['home_score'] < filtered_results['away_score']].groupby('home_team').size().reset_index(name='losses')

# Combine wins and losses into a single DataFrame
results_summary = pd.merge(wins, losses, left_on='home_team', right_on='home_team', how='outer').fillna(0)
results_summary['total_matches'] = results_summary['wins'] + results_summary['losses']
results_summary['win_frequency'] = results_summary['wins'] / results_summary['total_matches']

# Merge win frequency with the main DataFrame
df = df_teams.merge(team_stats, left_on='Team', right_on='home_team', how='left').drop(columns=['home_team'])
df = df.merge(defensive_rating, left_on='Team', right_on='home_team', how='left').drop(columns=['home_team'])
df = df.merge(offensive_rank, left_on='Team', right_on='home_team', how='left').drop(columns=['home_team'])
df = df.merge(results_summary[['home_team', 'win_frequency']], left_on='Team', right_on='home_team', how='left').drop(columns=['home_team'])

# Create the target variable based on win frequency
df['advanced'] = df['win_frequency'] > df['win_frequency'].median()  # Example threshold: median win frequency

# Convert target variable to binary
df['advanced'] = df['advanced'].astype(int)

# Features and target
features = ['avg_goals_scored', 'avg_goals_conceded', 'Defensive_Team_Rating', 'offensive_team_rank', 'win_frequency']
X = df[features]
y = df['advanced']

# Impute missing values
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Scale features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier(random_state=42)
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# Best Model Evaluation
best_rf = grid_search.best_estimator_
y_pred_proba = best_rf.predict_proba(X)[:, 1]  # Predict probabilities for all data
df['predicted_proba'] = y_pred_proba
df['predicted'] = (y_pred_proba > 0.5).astype(int)  # Apply threshold

# Post-processing to ensure constraints
grouped = df.groupby('Group')
selected_teams = pd.DataFrame()

# Ensure at least 2 teams from each group and no more than 3 teams
for group_name, group_df in grouped:
    # If the group has more than 3 teams, take the top 3 by probability
    if len(group_df) > 3:
        top_teams = group_df.sort_values(by='predicted_proba', ascending=False).head(3)
    else:
        top_teams = group_df.sort_values(by='predicted_proba', ascending=False)

    selected_teams = pd.concat([selected_teams, top_teams])

# Check if any group is missing
required_groups = df['Group'].unique()
missing_groups = [group for group in required_groups if group not in selected_teams['Group'].unique()]

# Ensure at least 2 teams from each group
for group in required_groups:
    if group not in selected_teams['Group'].unique():
        # If no team from this group is selected, add the top 2 teams from this group
        group_df = df[df['Group'] == group]
        additional_teams = group_df.sort_values(by='predicted_proba', ascending=False).head(2)
        selected_teams = pd.concat([selected_teams, additional_teams])

# Ensure exactly 32 teams advance
if len(selected_teams) > 32:
    selected_teams = selected_teams.sort_values(by='predicted_proba', ascending=False).head(32)
elif len(selected_teams) < 32:
    remaining_teams_needed = 32 - len(selected_teams)
    remaining_teams = df[~df.index.isin(selected_teams.index)]
    additional_teams = remaining_teams.sort_values(by='predicted_proba', ascending=False).head(remaining_teams_needed)
    selected_teams = pd.concat([selected_teams, additional_teams])

# Trim to exactly 32 teams if needed
selected_teams = selected_teams.drop_duplicates().head(32)

# Teams that advanced
teams_advanced = selected_teams[selected_teams['predicted'] == 1]

# Teams that did not advance
teams_not_advanced = df[~df.index.isin(selected_teams.index)]

# Print the selected teams
print(f'Accuracy: {accuracy_score(y_test, best_rf.predict(X_test))}')
print("Selected Teams:")
print(selected_teams)

Fitting 5 folds for each of 108 candidates, totalling 540 fits
Accuracy: 1.0
Selected Teams:
            Team    Group  avg_goals_scored  avg_goals_conceded  \
16         Spain  Group E          2.390863            0.730964   
34   Ivory Coast  Group I          2.041801            0.829582   
22         Ghana  Group F          1.951613            0.790323   
11          Iran  Group C          2.103550            0.665680   
10         Egypt  Group C          2.046512            0.841860   
25       Croatia  Group G          2.005495            0.868132   
14       Morocco  Group D          1.750000            0.610759   
2        Nigeria  Group A          1.927632            0.753289   
9         Brazil  Group C          2.463455            0.813953   
12     Argentina  Group D          2.226351            0.844595   
13       England  Group D          2.286778            0.878957   
29      Portugal  Group H          2.083565            0.902507   
4         Mexico  Group B          2

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score # Import metrics from sklearn.metrics
from sklearn.preprocessing import LabelEncoder

filtered_results = results[results['home_team'].isin(countries) | results['away_team'].isin(countries)]

filtered_results['date'] = pd.to_datetime(filtered_results['date'])
filtered_results = filtered_results[filtered_results['date'].dt.year > 2000]

le = LabelEncoder()

all_teams = pd.concat([filtered_results['home_team'], filtered_results['away_team']])
le.fit(all_teams)

filtered_results['home_team'] = le.transform(filtered_results['home_team'])
filtered_results['away_team'] = le.transform(filtered_results['away_team'])


<ipython-input-58-ae1c5bab7ba7>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_results['date'] = pd.to_datetime(filtered_results['date'])


In [ ]:
filtered_results

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
25070,2001-01-06,48,83,5,2,FIFA World Cup qualification,Miami,United States,True
25071,2001-01-06,60,204,2,1,Friendly,Cairo,Egypt,False
25076,2001-01-09,60,214,3,1,Friendly,Cairo,Egypt,False
25079,2001-01-12,2,32,2,1,African Cup of Nations qualification,Algiers,Algeria,False
25080,2001-01-12,26,16,2,0,Millennium Cup,Kochi,India,True
...,...,...,...,...,...,...,...,...,...
47394,2024-07-10,136,62,1,2,UEFA Euro,Dortmund,Germany,True
47395,2024-07-10,207,43,0,1,Copa América,Charlotte,United States,True
47396,2024-07-13,35,207,2,2,Copa América,Charlotte,United States,True
47397,2024-07-14,182,62,2,1,UEFA Euro,Berlin,Germany,True


In [ ]:
team_stats = {}

for team in np.unique(filtered_results[['home_team', 'away_team']].values):
    team_data = filtered_results[(filtered_results['home_team'] == team) | (filtered_results['away_team'] == team)]
    wins = len(team_data[((team_data['home_team'] == team) & (team_data['home_score'] > team_data['away_score'])) |
                         ((team_data['away_team'] == team) & (team_data['away_score'] > team_data['home_score']))])
    matches = len(team_data)
    goals_scored = team_data[team_data['home_team'] == team]['home_score'].sum() + team_data[team_data['away_team'] == team]['away_score'].sum()
    goals_conceded = team_data[team_data['home_team'] == team]['away_score'].sum() + team_data[team_data['away_team'] == team]['home_score'].sum()

    team_stats[team] = {
        'win_rate': wins / matches if matches > 0 else 0,
        'avg_goals_scored': goals_scored / matches if matches > 0 else 0,
        'avg_goals_conceded': goals_conceded / matches if matches > 0 else 0
    }

# Add features to the dataset
filtered_results['home_win_rate'] = filtered_results['home_team'].map(lambda x: team_stats[x]['win_rate'])
filtered_results['away_win_rate'] = filtered_results['away_team'].map(lambda x: team_stats[x]['win_rate'])
filtered_results['home_avg_goals_scored'] = filtered_results['home_team'].map(lambda x: team_stats[x]['avg_goals_scored'])
filtered_results['away_avg_goals_scored'] = filtered_results['away_team'].map(lambda x: team_stats[x]['avg_goals_scored'])
filtered_results['home_avg_goals_conceded'] = filtered_results['home_team'].map(lambda x: team_stats[x]['avg_goals_conceded'])
filtered_results['away_avg_goals_conceded'] = filtered_results['away_team'].map(lambda x: team_stats[x]['avg_goals_conceded'])

# Define target variable: home_win (1 if home_team wins, 0 otherwise)
filtered_results['home_win'] = (filtered_results['home_score'] > filtered_results['away_score']).astype(int)

# Select features and target variable
features = ['home_team', 'away_team', 'home_win_rate', 'away_win_rate', 'home_avg_goals_scored', 'away_avg_goals_scored', 'home_avg_goals_conceded', 'away_avg_goals_conceded']
X = filtered_results[features]
y = filtered_results['home_win']

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize and train the model
model = LogisticRegression()
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Display evaluation metrics
accuracy, precision, recall, f1


(0.7021276595744681,
 0.6803607214428857,
 0.6762948207171314,
 0.6783216783216782)

In [ ]:
import numpy as np
fifa_rankings = {
    "Argentina": 1, "France": 2, "Spain": 3, "England": 4, "Brazil": 5, "Belgium": 6, "Netherlands": 7,
    "Portugal": 8, "Colombia": 9, "Italy": 10, "Uruguay": 11, "Croatia": 12, "Germany": 13, "Morocco": 14,
    "Switzerland": 15, "USA": 16, "Mexico": 17, "Japan": 18, "Senegal": 19, "Iran": 20, "Denmark": 21,
    "Austria": 22, "South Korea": 23, "Australia": 24, "Ukraine": 25, "Turkey": 26, "Ecuador": 27,
    "Poland": 28, "Sweden": 29, "Wales": 30, "Hungary": 31, "Serbia": 32, "Russia": 33, "Qatar": 34,
    "Panama": 35, "Egypt": 36, "Venezuela": 37, "Ivory Coast": 38, "Nigeria": 39, "Canada": 40,
    "Tunisia": 41, "Peru": 42, "Czech Republic": 43, "Norway": 44, "Scotland": 45, "Bosnia and Herzegovina": 46,
    "Finland": 47, "Slovakia": 48, "Greece": 49, "Turkey": 50, "Romania": 51, "Israel": 52, "Iceland": 53,
    "Slovenia": 54, "North Macedonia": 55, "Bulgaria": 56, "Albania": 57, "Montenegro": 58, "Georgia": 59,
    "Armenia": 60, "Cyprus": 61, "Azerbaijan": 62, "Kosovo": 63, "Kazakhstan": 64, "Lithuania": 65,
    "Belarus": 66, "Luxembourg": 67, "Estonia": 68, "Faroe Islands": 69, "Malta": 70, "Moldova": 71,
    "Liechtenstein": 72, "Gibraltar": 73, "San Marino": 74, "Andorra": 75, "China PR": 87, "Cameroon": 51,"Honduras":79,
    "Mali": 52, "South Africa": 57, "Costa Rica": 46, "New Zealand": 122, "Algeria": 30, "United Arab Emirates": 69,
    "Iraq": 55,"Ghana": 64,"Saudi Arabia": 56
}
def calculate_team_strength(fifa_rankings):
    """
    Calculate a strength score for each team based on FIFA rankings.

    :param fifa_rankings: Dictionary with team names as keys and their FIFA rankings as values.
    :return: Dictionary with team names as keys and their strength scores as values.
    """
    strength_scores = {}

    max_rank = max(fifa_rankings.values())  # Assuming higher values are worse

    for team, rank in fifa_rankings.items():
        # Convert rank to a score (lower rank is better)
        score = 1 / rank  # Inverse rank as score
        strength_scores[team] = score

    return strength_scores
def simulate_match(team1, team2, team_probs, defensive_rank, goals_conceded):
    """
    Simulates a match between two teams based on their predicted probabilities,
    defensive rank, and goals conceded.

    :param team1: Name of the first team.
    :param team2: Name of the second team.
    :param team_probs: Dictionary with team names as keys and their winning probabilities as values.
    :param defensive_rank: Dictionary with team names as keys and their defensive ranks as values.
    :param goals_conceded: Dictionary with team names as keys and their goals conceded as values.
    :return: Winner of the match.
    """
    # Calculate strength scores
    strength_scores = calculate_team_strength(fifa_rankings)

    # Get the strength scores for the teams
    score_team1 = strength_scores.get(team1, 0.5)
    score_team2 = strength_scores.get(team2, 0.5)

    # Normalize scores to get probabilities
    total_score = score_team1 + score_team2
    prob_team1 = score_team1 / total_score
    prob_team2 = score_team2 / total_score

    # Simulate the match
    winner = team1 if np.random.rand() < prob_team1 else team2
    # print(f"Match: {team1} vs {team2} -> Winner: {winner}")

    return winner


# Example usage
# Define the matchups for the knockout stage (this should be based on group stage results)
def simulate_knockout_stage(teams_df, round):
    """
    Recursively simulates the knockout stage matches and determines the overall winner.

    :param teams_df: DataFrame of teams that advanced with their predicted probabilities,
                     defensive ranks, and goals conceded.
    :return: Name of the overall winner of the tournament.
    """
    # Extract team data
    team_probs = dict(zip(teams_df['Team'], teams_df['predicted_proba']))
    defensive_rank = dict(zip(teams_df['Team'], teams_df['Defensive_Team_Rating']))
    goals_conceded = dict(zip(teams_df['Team'], teams_df['avg_goals_conceded']))

    # Generate matchups for the current round
    def generate_knockout_matchups(teams):
        """
        Generate knockout stage matchups from the list of teams.

        :param teams: List of team names.
        :return: List of tuples, each containing two team names representing a matchup.
        """
        # Ensure the number of teams is even
        if len(teams) % 2 != 0:
            teams.append('Bye')  # Handle odd number of teams (Bye round)

        matchups = [(teams[i], teams[i + 1]) for i in range(0, len(teams), 2)]
        return matchups

    # Get the list of teams
    teams = teams_df['Team'].tolist()

    # Base case: If only one team is left, that team is the winner
    if len(teams) == 1:
        return teams[0]

    # Simulate the matches for the current round
    matchups = generate_knockout_matchups(teams)

    winners = []
    for team1, team2 in matchups:
        winner = simulate_match(team1, team2, team_probs, defensive_rank, goals_conceded)
        winners.append(winner)
        # print(f"{team1} vs {team2} -> Winner: {winner}")

    # Prepare DataFrame for the next round
    next_round_df = pd.DataFrame({'Team': winners})
    next_round_df = next_round_df.merge(teams_df[['Team', 'predicted_proba', 'Defensive_Team_Rating', 'avg_goals_conceded']], on='Team')

    # Recursively simulate the next round
    return simulate_knockout_stage(next_round_df,round+1)

# Example usage
# Assuming `selected_teams` is a DataFrame with the advanced teams, their probabilities,
# defensive ranks, and goals conceded
# You should have the selected_teams DataFrame from your previous code

# Simulate the knockout stage
win_frequency={}
for teams in countries:
  win_frequency[teams] = 0

for i in range(0,200):
  winner = simulate_knockout_stage(selected_teams,0)
  win_frequency[winner] += 1

print(win_frequency)

overall_winner = sorted([(k,v) for k, v in win_frequency.items()], key=lambda x: x[1], reverse=True)[0]

print(f'The overall winner of the tournament is {overall_winner}')

{'Germany': 3, 'France': 0, 'Spain': 48, 'England': 15, 'Italy': 2, 'Netherlands': 18, 'Portugal': 2, 'Belgium': 0, 'Croatia': 2, 'Denmark': 2, 'Switzerland': 0, 'Sweden': 0, 'Poland': 1, 'Austria': 0, 'Serbia': 0, 'Norway': 0, 'Brazil': 3, 'Argentina': 91, 'Uruguay': 6, 'Colombia': 0, 'Chile': 0, 'Ecuador': 0, 'Peru': 0, 'Mexico': 0, 'United States': 0, 'Canada': 0, 'Costa Rica': 0, 'Honduras': 1, 'Panama': 0, 'Nigeria': 0, 'Senegal': 1, 'Cameroon': 0, 'Egypt': 0, 'Ghana': 0, 'Morocco': 1, 'Algeria': 1, 'Tunisia': 0, 'Ivory Coast': 1, 'Japan': 0, 'South Korea': 0, 'Iran': 2, 'Australia': 0, 'Saudi Arabia': 0, 'Qatar': 0, 'United Arab Emirates': 0, 'Iraq': 0, 'China PR': 0, 'New Zealand': 0, 'Bosnia and Herzegovina': 0}
The overall winner of the tournament is ('Argentina', 91)


In [ ]:
# Example prediction

# Convert team names to numerical values

def run_match(team_1, team_2):
  new_match = {
      'home_team': le.transform([team_1])[0],
      'away_team': le.transform([team_2])[0],
      'home_win_rate': team_stats[le.transform([team_1])[0]]['win_rate'],
      'away_win_rate': team_stats[le.transform([team_2])[0]]['win_rate'],
      'home_avg_goals_scored': team_stats[le.transform([team_1])[0]]['avg_goals_scored'],
      'away_avg_goals_scored': team_stats[le.transform([team_2])[0]]['avg_goals_scored'],
      'home_avg_goals_conceded': team_stats[le.transform([team_1])[0]]['avg_goals_conceded'],
      'away_avg_goals_conceded': team_stats[le.transform([team_2])[0]]['avg_goals_conceded']
  }

  # Convert to DataFrame
  new_match_df = pd.DataFrame([new_match])

  # Predict outcome
  predicted_outcome = model.predict(new_match_df)
  if predicted_outcome == 1:
    return team_1
  else:
    return team_2

run_match('Germany', 'France')

NameError: name 'le' is not defined

In [ ]:
def run_group(group):

  group_wins = {}
  for i in group:
    group_wins[i] = 0

  for i in group:
    for j in group:
      if i != j:
        group_wins[run_match(i, j)] += 1

  return group_wins

def run_all_groups(groups):
  all_winning_teams = {}
  losers_og_vals = {}
  losers_group = []
  for i in groups.keys():
    group_dict = run_group(groups[i])
    group_pairs = {(k, v) for k, v in group_dict.items()}
    group_pairs = sorted(group_pairs, key=lambda item: item[1], reverse=True)
    all_winning_teams[group_pairs[0][0]] = group_pairs[0][1]
    all_winning_teams[group_pairs[1][0]] = group_pairs[1][1]
    losers_og_vals[group_pairs[2][0]] = group_pairs[2][1]
    losers_group.append(group_pairs[2][0])

  losers_results = run_group(losers_group)
  losers_pairs = {(k, v) for k, v in losers_results.items()}
  losers_pairs = sorted(losers_pairs, key=lambda item: item[1], reverse=True)

  print(len(all_winning_teams))

  for i in range(8):
    all_winning_teams[losers_pairs[i][0]] = losers_og_vals[losers_pairs[i][0]]

  print(len(all_winning_teams))

  return all_winning_teams

def make_seeds(team_pairs):
  seeds = {(k, v) for k, v in team_pairs.items()}
  seeds = sorted(seeds, key=lambda item: item[1], reverse=True)
  seeds = [k for k, v in seeds]
  return seeds

def play_seeds(seeds, round):
  new_seeds = []
  print(f"Round {round}")
  if len(seeds) % 2 == 1:
    new_seeds.append(seeds.pop(0))
  for i in range((len(seeds))//2):
    new_seeds.append(run_match(seeds[i], seeds[len(seeds)-1-i]))
    print(f"{seeds[i]} vs {seeds[len(seeds)-1-i]} is {new_seeds[-1]}")
  if len(new_seeds) > 1:
    return play_seeds(new_seeds, round+1)
  else:
    return new_seeds

play_seeds(make_seeds(run_all_groups(groups)), 0)


24
32
Round 0
Netherlands vs South Africa is Netherlands
Poland vs Mexico is Mexico
Spain vs Qatar is Spain
Sweden vs Cameroon is Cameroon
France vs Colombia is France
Brazil vs Egypt is Brazil
Portugal vs Tunisia is Tunisia
Belgium vs Bosnia and Herzegovina is Belgium
Italy vs United States is United States
Croatia vs England is England
Algeria vs Germany is Germany
Ecuador vs Argentina is Argentina
Iran vs Senegal is Senegal
Nigeria vs Ivory Coast is Ivory Coast
Morocco vs Switzerland is Morocco
Costa Rica vs Denmark is Denmark
Round 1
Netherlands vs Denmark is Denmark
Mexico vs Morocco is Morocco
Spain vs Ivory Coast is Spain
Cameroon vs Senegal is Senegal
France vs Argentina is Argentina
Brazil vs Germany is Germany
Tunisia vs England is England
Belgium vs United States is United States
Round 2
Denmark vs United States is United States
Morocco vs England is England
Spain vs Germany is Germany
Senegal vs Argentina is Argentina
Round 3
United States vs Argentina is Argentina
England 

['Germany']

In [ ]:
def predict_shootout_winner(shootouts, country1, country2):

    # Create a binary target variable: 1 if home_team wins, 0 if away_team wins
    filtered_shootouts['winner'] = (filtered_shootouts['winner'] == filtered_shootouts['home_team']).astype(int)

    # Prepare data for classification model
    X = filtered_shootouts[['home_team', 'away_team']]  # Features
    y = filtered_shootouts['winner']  # Target

    # Encode categorical variables
    X_encoded = pd.get_dummies(X, columns=['home_team', 'away_team'])

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)

    # Train a classification model
    classifier = RandomForestClassifier(n_estimators=100, random_state=42)
    classifier.fit(X_train, y_train)

    # Predict and evaluate the classification model
    y_pred = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {accuracy}')

    # Create a DataFrame for the specific shootout with all necessary columns
    match = pd.DataFrame(0, index=[0], columns=X_encoded.columns)  # Initialize with all columns and zeros
    match['home_team_' + country1] = 1
    match['away_team_' + country2] = 1

    # Ensure that the DataFrame has all necessary columns (including dummy variables for both teams)
    for col in X_encoded.columns:
        if col not in match.columns:
            match[col] = 0

    # Predict the winner
    prediction = classifier.predict(match)
    winner = country1 if prediction[0] == 1 else country2
    return f"The winner of the shootout between {country1} and {country2} is {winner}."

    value = predict_shootout_winner('Mexico','Colombia')
    print(value)